# Converting Arepo snapshots to be usable by TARDIS

In [ ]:
from tardis.io.parsers import arepo
import json

As a first step, the relevant data has to be loaded from an Arepo snapshot. In case you have the arepo-snap-util package installed, you can use the built in wrapper (as described below) to load the relevant data. In case you do not have this package installed or want to load the snapshot in a different way, you can manually provide the relevant data and continue with the next step.

If you're using the built-in tool, you will need to provide the path to the snapshot file, a list with the elements you want to include in your Tardis model, as well as the species file with which the Arepo simulation was run.

In [ ]:
try:
    snapshot = arepo.ArepoSnapshot("arepo_snapshot.hdf5", ["ni56", "si28"], "species55.txt", resolution=32)
    pos, vel, rho, xnuc, time = snapshot.get_grids()
except ImportError:
    print("Looks like arepo-snap-util is missing!")

This will load the necessary from the snapshot. See the API documentation for more options on how to load snapshots.

As you can see, it will fail if you do not have the arepo-snap-util package installed. So lets manually load the data. In this case from a `json` file, where the data which would have been loaded with the function above has been manually saved to.

In [ ]:
with open('arepo_snapshot.json') as json_file:
    data = json.loads(json.load(json_file))
    
pos, vel, rho, xnuc, time = data["pos"], data["vel"], data["rho"], data["xnuc"], data["time"]

print("Position data shape: ", pos.shape)
print("Velocity data shape: ", vel.shape)
print("Density data shape: ", rho.shape)
print("Nuclear data shape (per element): ", xnuc["ni56"].shape)

In case you want to load the snapshot data itself with your own tools, you will need to provide the following data:
- Position in the center of mass frame ("pos") -> `np.array`
- Velocity ("vel") -> `np.array`
- Density ("rho") -> `np.array`
- Nuclear fraction ("xnuc") of each element you want to include -> `dict` containing `np.array`   
- Time of the snapshot ("time") -> `float`

The data is expected to be mapped to a Carthesian grid and should have the same shape as the one provided by the built-in tool.

Now You can create a TARDIS model. There are three possibilities on how to extract the profiles from the snapshot:
- **Line profile**: This extracts the data along a straight line (the x-axis) without any averaging
- **Cone profile**: This extracts the data within a specified cone *(TBD)*
- **Full profile**: This averages over the whole simulation *(TBD)*

In [ ]:
profile = arepo.LineProfile(pos, vel, rho, xnuc, time)

This loads the data (here for the line profile), which can then be cut to the ranges which you want to include in your TARDIS model. The syntax for the other profiles is similar:
- `arepo.ConeProfile(<args>)`
- `arepo.FullProfile(<args>)`

Next you can create the profiles acccording to the model option you selected. A diagnostic plot will be shown per default, but this behaviour can be turned off with the option `show_plot=False`.

In [ ]:
profile.create_profile()

In many cases you only want a very specific region from the snapshot, e.g. cutting out the dense, optically thick regions. This can be acchieved using the keywords `inner_radius` and `outer_radius`.

In [ ]:
profile.create_profile(inner_radius=1e11, outer_radius=2e11)

Once you have created a profile of the desired region, you can export the profile to a `csvy` file, which in turn can be used in a TARDIS model. Here you have to specify how many shells you want to export. Note that the number of exported shells cannot be larger than the resolution of the given grid since no interpolation on the profile is done. If you want more shells, you either need to map your snapshot to a higher resolution Carthesian grid or interpolate the initial data yourself.

In [ ]:
profile.export(5, "snapshot_converted_to_tardis.csvy")

During the export, the `yaml` header is automatically written and includes the time of the screenshot as the time for both the nuclear data as well as the density profile.

All abundences will normalised such that roughly sum to 1, but slight deviations are expected to occur.

Note that the export function will not overwrite existing files, but rather add an incrementing number to the filename.